In [1]:
import sys
import os

In [2]:
import pandas as pd
import numpy as np

In [3]:
build_project_path = os.environ['BUILD_PROJECT_PATH']

In [4]:
synthetic_data_path = os.path.join(build_project_path, 'synthetic_data')
data_path = os.path.join(synthetic_data_path, 'data')

In [5]:
if synthetic_data_path not in sys.path:
    sys.path.append(synthetic_data_path)

In [6]:
from llm_requests import generate_prompt

jitter_df = pd.read_csv(os.path.join(data_path, 'jitter_ds.csv'))

seed_titles = jitter_df['clean_title'].unique()

example_seed_titles = np.random.choice(seed_titles, 5)

example_prompt = generate_prompt(example_seed_titles)

/home/jamesa261/fine-tuning-build-project/synthetic_data/llm_requests.py:8: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [19]:
model_name = 'Meta-Llama-3.3-70B-Instruct'
# model_name = 'Meta-Llama-3.1-70B-Instruct'

In [20]:
from llm_requests import get_client

client = get_client()

In [21]:
from llm_requests import async_make_api_call

example_response = await async_make_api_call(client, model_name, example_prompt, perturbation_std=0.1)

In [22]:
print(example_response.choices[0].message.content)

<response>
1. [`Ranch Hand - Cattle Operation (Live-In Position)`, `Seasonal Ranch Hand Needed - Horse Farm in Montana`, `Ranch Hand / Farm Laborer - Full-Time, Year-Round`, `Entry-Level Ranch Hand Opportunity - Training Provided`, `Experienced Ranch Hand Wanted - Large Ranch in Texas`]
2. [`Senior Public Health Epidemiologist - Infectious Disease`, `Public Health Epidemiologist (Part-Time) - COVID-19 Response`, `Epidemiologist - Public Health (Temporary Position)`, `Public Health Epidemiologist - Maternal & Child Health`, `Public Health Epidemiologist - Global Health Initiative (Remote)`]
3. [`Speech and Language Specialist - Pediatric Clinic (Full-Time)`, `Speech-Language Pathologist (SLP) - School Setting`, `Bilingual Speech and Language Specialist - Early Intervention`, `Speech and Language Specialist - Rehabilitation Hospital`, `Lead Speech and Language Specialist - Autism Center`]
4. [`Technical Illustrator - 3D Modeling & Animation (Contract)`, `Senior Technical Illustrator - Ae